## Imports

In [2]:
import time

import numpy as np
import cv2 as cv
import os
import glob
import matplotlib.pyplot as plt
from numpy.random import uniform
import pdb

## Show functions

In [15]:
def show_image_cv(title,image,fx=0.2,fy=0.2):
    image = cv.resize(image,(0,0),fx=fx,fy=fy)
    cv.imshow(title, image)
    cv.waitKey(0)
    cv.destroyAllWindows()

def show_image_matplot(title, image):
    image_resized = cv.resize(image, (0, 0), fx=0.3, fy=0.3)

    image = cv.cvtColor(image_resized, cv.COLOR_BGR2RGB)

    plt.imshow(image)
    plt.title(title)
    plt.axis('off')  # Ascunde axele pentru a afișa doar imaginea
    plt.show()

In [16]:
img = cv.imread("antrenare/1_01.jpg")
show_image_cv("test",img)

In [ ]:
# hsv mask
lower_hsv_bound = np.array([20,0,0])
upper_hsv_bound = np.array([255,255,255])
hsv_image = cv.cvtColor(img, cv.COLOR_BGR2HSV)

mask = cv.inRange(hsv_image, lower_hsv_bound, upper_hsv_bound)
show_image_cv("mask",mask)

new_image = cv.bitwise_and(img, img, mask=mask)
show_image_cv("ceva",new_image)

# Get contours function

In [41]:
def get_countours(original_image,masked_image_grey):
    edges = cv.Canny(masked_image_grey, 50, 200)
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    show_image_cv("edges",edges)

    max_area = 0
    for i in range(len(contours)):
            if(len(contours[i]) >3):
                possible_top_left = None
                possible_bottom_right = None
                for point in contours[i].squeeze():
                    if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                        possible_top_left = point

                    if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                        possible_bottom_right = point

                diff = np.diff(contours[i].squeeze(), axis = 1)
                possible_top_right = contours[i].squeeze()[np.argmin(diff)]
                possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
                if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                    max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                    top_left = possible_top_left
                    bottom_right = possible_bottom_right
                    top_right = possible_top_right
                    bottom_left = possible_bottom_left

    width = 810
    height = 810

    image_copy = cv.cvtColor(masked_image_grey.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(image_copy,tuple(top_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(top_right),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(image_copy,tuple(bottom_right),20,(0,0,255),-1)
    show_image_cv("detected corners",image_copy)

    img_grey = cv.cvtColor(img.copy(),cv.COLOR_BGR2GRAY)

    puzzle = np.array([top_left,top_right,bottom_left,bottom_right],dtype=np.float32)
    destination = np.array([[0,0],[width,0],[0,height],[width,height]],dtype=np.float32)
    M = cv.getPerspectiveTransform(puzzle,destination)
    result = cv.warpPerspective(img_grey,M,(width,height))
    result = cv.cvtColor(result,cv.COLOR_GRAY2BGR)

    return result

In [42]:
# canny for edge detection

new_image_gray = cv.cvtColor(new_image, cv.COLOR_BGR2GRAY)
result = get_countours(img,new_image_gray)


In [44]:
show_image_cv("result",result,fx=1,fy=1)